In [ ]:
import numpy as np
from pandas import read_csv, DataFrame
from keras.datasets import mnist
import matplotlib.pyplot as plt
from tensorflow.keras import utils
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,Dropout
from tensorflow.keras.preprocessing import image
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler 
import seaborn as sns
from sklearn.model_selection import train_test_split


url = r"C:/bauman/git_bauman/data/ebw_data.csv"
dataset = pd.read_csv(url)
#dataset.head()
#dataset.describe()

# посмотрели есть ли выпадения данных
print('percentage of NaN cells:')
print(dataset.isna().sum() / dataset.shape[0] *100)


# нормализация и запоминание мин и макс по столбцам
maxdataset = dataset[['IW','IF','VW','FP','Depth','Width']].max()
mindataset = dataset[['IW','IF','VW','FP','Depth','Width']].min()

maxIW = maxdataset['IW'] 
minIW = mindataset['IW'] 
normIW = (dataset['IW'] - minIW ) / (maxIW - minIW )

maxIF = maxdataset['IF'] 
minIF = mindataset['IF'] 
normIF = (dataset['IF'] - minIF ) / (maxIF - minIF )

maxVW = maxdataset['VW'] 
minVW = mindataset['VW'] 
normVW = (dataset['VW'] - minVW ) / (maxVW - minVW )

maxFP = maxdataset['FP'] 
minFP = mindataset['FP'] 
normFP = (dataset['FP'] - minFP ) / (maxFP - minFP )

maxDepth = maxdataset['Depth'] 
minDepth = mindataset['Depth'] 
normDepth = (dataset['Depth'] - minDepth ) / (maxDepth - minDepth )

maxWidth = maxdataset['Width'] 
minWidth = mindataset['Width'] 
normWith = (dataset['Width'] - minWidth ) / (maxWidth - minWidth )

normdataset = pd.DataFrame({'IW': normIW, 
                            'IF': normIF,
                            'VW': normVW,
                            'FP': normFP,
                            'Depth': normDepth,
                            'Width': normWith,
                            })

# переменные для входных и выходных данных
X = normdataset[['IW','IF','VW','FP']]
Y = normdataset[['Depth','Width']] 

# делим выборку и перемешиваем
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size = 0.05, shuffle = True)

# создаем модель
model = Sequential()
model.add(Dense(4, activation = 'sigmoid'))
model.add(Dense(2, activation = 'sigmoid'))
model.compile( loss = 'mse',optimizer = 'adam', metrics = ['accuracy'] )

history = model.fit(X_train,Y_train, epochs = 1000, validation_data = (X_train,Y_train), verbose = 1)

# для инфомации выводит кривую обучения
plt.plot(history.history['accuracy'],label = 'Обучающая выборка')
plt.plot(history.history['val_accuracy'],label = 'Точность тест')
plt.title('Обучение')
plt.xlabel('эпохи')
plt.ylabel(model.loss)

In [67]:
# проверка ввода пользователя, используем одну из строчек из dataset 
# 
# 47	134	10.0	110	0.92	1.86
human_input = pd.DataFrame({'IW': [47],
                            'IF': [134],
                            'VW': [10],
                            'FP': [110]})

# нормализуем введеные данные
human_input['IW'] = (human_input['IW'] - minIW) / (maxIW - minIW)
human_input['IF'] = (human_input['IF'] - minIF) / (maxIF - minIF)
human_input['VW'] = (human_input['VW'] - minVW) / (maxVW - minVW)
human_input['FP'] = (human_input['FP'] - minFP) / (maxFP - minFP)

In [68]:
# делаем предсказание
mypredcit = model.predict(human_input)

predict = pd.DataFrame({'Depth': mypredcit[:,0]*(maxDepth-minDepth)+minDepth, 
                        'Width': mypredcit[:,1]*(maxWidth-minWidth)+minWidth})

# тестовые значение для ответа: 0.92	1.86
predict

1/1 [==============================] - 0s 55ms/step


,Depth,Width
0,0.993897,2.000333
